Jyoti Chandrawanshi<br>
Term Project<br>
Etsy Women Clothing Products<br>
Tools For Data Analysis<br>

In [40]:
# Import important libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [41]:
def readetsy():
    dfs = []
    clothitem = ["dresses", "tops-and-tees", "skirts", "jackets-and-coats", "pants-and-capris", "sweaters", "swimwear"]
    for Citem in clothitem:
        print(Citem)
        for i in range(1, 11):
            url = 'https://www.etsy.com/c/clothing/womens-clothing/'+str(Citem)+'?ref=pagination&page='+str(i)+''
            etsyweb = requests.get(url).text
            soup = BeautifulSoup(etsyweb)
            df = readpage(soup, Citem)
            dfs.append(df)
    
        etsydf = pd.concat(dfs, ignore_index=True)
    
    return etsydf

In [42]:
def readpage(soup, Citem):
    name = []
    merchant = []
    ratings = []
    users = []
    cost = []
    orgcost = []
    shipping = []
    Cl_Type = []
    x=0
    for row in soup.select('.responsive-listing-grid'):
        for item in row.select('.wt-list-unstyled'):
            x+=1
            #print(x)
            name.append(item.select('h3')[0].get_text().strip())
            #print('=>', item.select('h3')[0].get_text().strip())
            merchant.append(item.select('.text-gray-lighter')[0].get_text().strip())
            
            for rating in item.select('.v2-listing-card__rating'):  
                #print(rating)
                try:
                    ratings.append(item.input['value'])
                except:
                    ratings.append(np.nan)
                
                try:
                    users.append(rating.select('.icon-b-1')[0].get_text().strip())
                except:
                    users.append(np.nan)
        
            for price in item.select('.n-listing-card__price'):
                cost.append(price.select('.currency-value')[0].get_text().strip())
                try:
                    orgcost.append(price.select('.currency-value')[1].get_text().strip())
                except:
                    orgcost.append(np.nan)
            
            try:
                promo = item.select('.wt-badge')[0].get_text().strip()
                shipping.append(promo)
            except:
                shipping.append(np.nan)
            
            Cl_Type.append(Citem)
        
    df = pd.DataFrame({"Item": name, "Category": Cl_Type, "Seller": merchant, "Ratings": ratings, "Ucount": users, "Price": cost, "OrgPrice": orgcost,
                       "Promo": shipping})
    
    return df

In [43]:
#dressdf = readetsy()
#dressdf.info()
#dressdf.to_csv('dressdf.csv')

In [44]:
test = pd.read_csv('dressdf.csv')

In [45]:
test.head()

,Unnamed: 0,Item,Category,Seller,Ratings,Ucount,Price,OrgPrice,Promo
0,0,"Purple Cheongsam Cocktail dress, High neck qip...",dresses,TAVROVSKA,4.9500,(325),79.89,NaN,NaN
1,1,"Festival Dress, Boho Dress, Pixie Dress, Fairy...",dresses,NatureSpiritDesigns,4.9230,"(3,626)",65.00,NaN,FREE shipping
2,2,"Grey Bridesmaid Dress,Long Lace Chiffon Simple...",dresses,NobleGown,4.8000,(200),129.00,NaN,FREE shipping
3,3,Women's linen Short Sleeves Summer maxi dress ...,dresses,Youga,4.7325,"(3,083)",52.42,69.90,FREE shipping
4,4,"Women's Black Tunic, Black Women Kaftan, Overs...",dresses,SHIHAR,4.8897,"(1,875)",124.97,147.03,NaN


In [46]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 0 to 3359
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3360 non-null   int64  
 1   Item        3360 non-null   object 
 2   Category    3360 non-null   object 
 3   Seller      3360 non-null   object 
 4   Ratings     3259 non-null   float64
 5   Ucount      3259 non-null   object 
 6   Price       3360 non-null   object 
 7   OrgPrice    568 non-null    object 
 8   Promo       1681 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 236.4+ KB


In [47]:
# Replace all the Category to numbers 
test['Category'] = test['Category'].replace(["dresses","tops-and-tees","skirts", "jackets-and-coats", "pants-and-capris", "sweaters", "swimwear"],[1,2,3,4,5,6,7])


In [48]:
test["Promo"] = test["Promo"].replace(["FREE shipping","Bestseller",np.nan],[1,0,0])

In [49]:
test.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 0 to 3359
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3360 non-null   int64  
 1   Item        3360 non-null   object 
 2   Category    3360 non-null   int64  
 3   Seller      3360 non-null   object 
 4   Ratings     3259 non-null   float64
 5   Ucount      3259 non-null   object 
 6   Price       3360 non-null   object 
 7   OrgPrice    568 non-null    object 
 8   Promo       3360 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 236.4+ KB


In [50]:
# Remove etra column which i am not using
test1 = test.drop(columns=["Item","Seller","Ucount","OrgPrice",])

In [51]:
# Drop null value rows 
test2 =test1.dropna() 

In [61]:
#Remove unnamed column
test3 = test2.loc[:, ~test2.columns.str.contains('^Unnamed')]
#test3['Price'].astype(str)
#test3["Price"].apply(lambda x: x.str.replace(',',''))
#test3['Price'] = test3['Price'].astype(float)
test4 = requests.get()

AttributeError: 'str' object has no attribute 'str'

In [53]:
reg = smf.ols("Price ~ Promo + Ratings + Category", data =test3).fit()
print(reg.summary())

ValueError: endog has evaluated to an array with multiple columns that has shape (3259, 1170). This occurs when the variable converted to endog is non-numeric (e.g., bool or str).

In [ ]:
te